In [1]:
import pyscf
import numpy as np
import h5py
import time
import scipy
import itertools

In [2]:
MACHEPS = 1e-12

def set_bit(bit_loc):
    """
    Set the bit_loc-th bit in bit string f. Returns unchanged if the bit is already set. bit_loc is zero-indexed.
    """
    f = 0
    for loc in bit_loc:
        f = f | 1<<loc
    return f

def set_bit_single(f, bit_loc):
    """
    Set the bit_loc-th bit in bit string f. Returns unchanged if the bit is already set. bit_loc is zero-indexed.
    """
    return f | 1<<bit_loc

def clear_bit(f, bit_loc):
    """
    Unset the bit_loc-th bit in bit string f. Returns unchanged if the bit is already unset. bit_loc is zero-indexed.
    """
    return f & ~(1<<bit_loc)

def test_bit(f, bit_loc):
    """
    Test if bit_loc in f is set. Returns 1 if set, 0 if not set.
    """
    return (f & (1<<bit_loc)) >> bit_loc


def count_set_bits(f):
    """
    Return the number of set (1) bits in the bit string f.
    """
    return int(bin(f).count('1'))

def get_excitation_level(f1, f2):
    """
    Get the excitation level between two bit strings f1 and f2, i.e., half the Hamming distance.
    """
    return int(count_set_bits(f1^f2)/2)

def test_bit(f, bit_loc):
    """
    Test if bit_loc in f is set. Returns 1 if set, 0 if not set.
    """
    return (f & (1<<bit_loc)) >> bit_loc

def annop(bit_string, ispinor):
        """
        Annihilation operator, annihilates spinorb in bit_string, returns the sign and the resulting bit string.
        If spinorb is already empty, sign is zero and the bit string is returned unchanged.
        """
        if (not test_bit(bit_string, ispinor)):
            sgn = 0
        else:
            test_string = 0
            for i in range(ispinor):
                test_string = set_bit_single(test_string, i)
            sgn = (-1)**(count_set_bits(bit_string & test_string))
            bit_string = clear_bit(bit_string, ispinor)
        return (sgn, bit_string)
    
def bstring_to_occ_vec(f, nelec, norbs):
    occ_vec = np.zeros(nelec, dtype='int')
    nfound = 0
    for i in range(norbs):
        if test_bit(f, i)==1:
            occ_vec[nfound] = i
            nfound += 1
            if (nfound==nelec):
                break
    return occ_vec

def get_excit_connection(f1, f2, exlvl, nelec, norbs):
    excit_bstring = f1^f2
    
    excit = np.zeros((2,exlvl), dtype='int')
    nbit_f1_found = 0
    nbit_f2_found = 0
    for i in range(norbs):
        if (test_bit(excit_bstring, i)==1):
            # Check where this electron is coming from / going to
            if (test_bit(f1, i)):
                excit[0][nbit_f1_found] = i
                nbit_f1_found += 1
            else:
                excit[1][nbit_f2_found] = i
                nbit_f2_found += 1
            if (nbit_f1_found == exlvl and nbit_f2_found==exlvl):
                break
                
    # Get permutation!
    perm = annop_mult(f1, excit[0])[0] * annop_mult(f2, excit[1])[0]

    return excit, perm

def annop_mult(f, orbs):
    fold = f
    perm = 1
    for orb in orbs:
        iperm, fnew = annop(fold, orb)
        perm *= iperm
        fold = fnew
    
    return perm, fnew

def make_cumulants(rdm):
    try:
        assert rdm['max_rdm_level'] >= 2
    except AssertionError:
        raise Exception('Max RDM level is 1, cumulants not necessary!')
        
    _lamb = {'max_cumulant_level':rdm['max_rdm_level']}
    _lamb['gamma1'] = rdm['1rdm']
    _lamb['eta1'] = -rdm['1rdm']+1
    _lamb['lambda2'] = rdm['2rdm'] - np.einsum('pr,qs->pqrs', rdm['1rdm'], rdm['1rdm']) + np.einsum('ps,qr->pqrs', rdm['1rdm'], rdm['1rdm'])
    if (rdm['max_rdm_level'] == 3):
        _lamb['lambda3'] = rdm['3rdm'] 
        - np.einsum('ps,qrtu->pqrstu',rdm['1rdm'],rdm['2rdm']) + np.einsum('pt,qrsu->pqrstu',rdm['1rdm'],rdm['2rdm']) + np.einsum('pu,qrts->pqrstu',rdm['1rdm'],rdm['2rdm'])
        - np.einsum('qt,prsu->pqrstu',rdm['1rdm'],rdm['2rdm']) + np.einsum('qs,prtu->pqrstu',rdm['1rdm'],rdm['2rdm']) + np.einsum('qu,prst->pqrstu',rdm['1rdm'],rdm['2rdm'])
        - np.einsum('ru,pqst->pqrstu',rdm['1rdm'],rdm['2rdm']) + np.einsum('rs,pqut->pqrstu',rdm['1rdm'],rdm['2rdm']) + np.einsum('rt,pqsu->pqrstu',rdm['1rdm'],rdm['2rdm'])
        + 2*(np.einsum('ps,qt,ru->pqrstu',rdm['1rdm'],rdm['1rdm'],rdm['1rdm']) + np.einsum('pt,qu,rs->pqrstu',rdm['1rdm'],rdm['1rdm'],rdm['1rdm']) + np.einsum('pu,qs,rt->pqrstu',rdm['1rdm'],rdm['1rdm'],rdm['1rdm']))
        - 2*(np.einsum('ps,qu,rt->pqrstu',rdm['1rdm'],rdm['1rdm'],rdm['1rdm']) + np.einsum('pu,qt,rs->pqrstu',rdm['1rdm'],rdm['1rdm'],rdm['1rdm']) + np.einsum('pt,qs,ru->pqrstu',rdm['1rdm'],rdm['1rdm'],rdm['1rdm']))
                
    return _lamb

class mySCF:
    def __init__(self, mol, c0=None):
        self.mol = mol
        self.nuclear_repulsion = self.mol.energy_nuc()
        self.nelec = sum(self.mol.nelec)
        if (c0 is None):
            self.c0 = pyscf.lib.param.LIGHT_SPEED
        else:
            self.c0 = c0
            #pyscf.lib.param.LIGHT_SPEED = c0
        
    def run_rhf(self):
        self.rhf = pyscf.scf.RHF(self.mol)
        self.rhf_energy = self.rhf.kernel()
        print(f"Non-relativistic RHF Energy: {self.rhf_energy:15.7f} Eh")
        
        #self.rhf_hcore_ao = self.mol.intor_symmetric('int1e_kin') + self.mol.intor_symmetric('int1e_nuc')
        #self.rhf_hcore_mo = np.einsum('pi,pq,qj->ij', self.rhf.mo_coeff, self.rhf_hcore_ao, self.rhf.mo_coeff)
        #self.rhf_eri_ao = self.mol.intor('int2e_sph') # Chemist's notation (ij|kl)
        #self.rhf_eri_mo = ao2mo.incore.full(self.rhf_eri_ao, self.rhf.mo_coeff)
        
    def run_dhf(self, rebuild=False, debug=False):
        # Run relativistic Dirac-Hartree-Fock
        self.dhf = pyscf.scf.DHF(self.mol)
        self.dhf.with_gaunt = False
        self.dhf.with_breit = False
        self.dhf.with_ssss = True
        # Add additional corrections here if desired
        self.dhf_energy = self.dhf.kernel()
        print(f"Relativistic DHF Energy:     {self.dhf_energy:15.7f} Eh")
        
        if (rebuild):
            _t0 = time.time()
            self.norb = len(self.dhf.mo_energy)
            self.nlrg = int(.5*len(self.dhf.mo_energy))
            self.nocc = sum(self.mol.nelec)
            self.nvirt = self.nlrg - self.nocc
            
            _t1 = time.time()
            # Harvest h from DHF (Includes both S & L blocks)
            self.dhf_hcore_ao = self.dhf.get_hcore()
            self.dhf_hcore_mo = np.einsum('pi,pq,qj->ij', np.conjugate(self.dhf.mo_coeff[:,self.nlrg:]), self.dhf_hcore_ao, self.dhf.mo_coeff[:,self.nlrg:])

            self.dhf_eri_ao_SSLL = self.mol.intor('int2e_spsp1_spinor')/(2*self.c0)**2 # kinetic balance

            self.dhf_eri_ao = np.zeros((self.norb,self.norb,self.norb,self.norb),dtype='complex128')
            self.dhf_eri_ao[self.nlrg:,self.nlrg:,self.nlrg:,self.nlrg:] = self.mol.intor('int2e_spsp1spsp2_spinor')/(2*self.c0)**4
            self.dhf_eri_ao[:self.nlrg,:self.nlrg,:self.nlrg,:self.nlrg] = self.mol.intor('int2e_spinor')
            self.dhf_eri_ao[self.nlrg:,self.nlrg:,:self.nlrg,:self.nlrg] = self.dhf_eri_ao_SSLL
            self.dhf_eri_ao[:self.nlrg,:self.nlrg,self.nlrg:,self.nlrg:] = (self.dhf_eri_ao_SSLL.swapaxes(0,2).swapaxes(1,3)) # No need to conjugate as we're using Chemist's notation

            _t2 = time.time()
            
            self.dhf_eri_mo_full = np.einsum('pi,qj,pqrs,rk,sl->ijkl',np.conjugate(self.dhf.mo_coeff[:,self.nlrg:]),(self.dhf.mo_coeff[:,self.nlrg:]),self.dhf_eri_ao,np.conjugate(self.dhf.mo_coeff[:,self.nlrg:]),(self.dhf.mo_coeff[:,self.nlrg:]),optimize=True)
            
            _t3 = time.time()
            
            self.dhf_eri_full_asym = self.dhf_eri_mo_full.swapaxes(1,2) - self.dhf_eri_mo_full.swapaxes(1,2).swapaxes(2,3)
            self.dhf_e1 = np.einsum('ii->',self.dhf_hcore_mo[:self.nocc, :self.nocc])
            self.dhf_e2 = 0.5*np.einsum('ijij->',self.dhf_eri_full_asym[:self.nocc, :self.nocc, :self.nocc, :self.nocc])            
            self.dhf_e_rebuilt = self.dhf_e1 + self.dhf_e2 + self.nuclear_repulsion
            if (debug):
                print(f"Rebuilt DHF Energy:          {self.dhf_e_rebuilt.real:15.7f} Eh")
                print(f"Diff:           {np.abs(self.dhf_e_rebuilt.real - self.dhf_energy):.6e}")
                print(f"Diff 1e:        {np.abs(self.dhf.scf_summary['e1']-self.dhf_e1):.6e}")
                print(f"Diff 2e:        {np.abs(self.dhf.scf_summary['e2']-self.dhf_e2):.6e}")
            
            _t4 = time.time()
            
            print(f'\nTiming report')
            print(f'....integral retrieval:      {(_t2-_t1):15.7f} s')
            print(f'....integral transformation: {(_t3-_t2):15.7f} s')
            print(f'....integral contractions:   {(_t4-_t3):15.7f} s')
            print(f'Total time taken:            {(_t4-_t0):15.7f} s\n')
    
    def run_mp2(self):
        _t0 = time.time()
        _upq = np.einsum('piqi->pq',self.dhf_eri_full_asym[:,:self.nocc,:,:self.nocc])
        self.dhf_fock_mo = _upq + self.dhf_hcore_mo

        self.D2 = np.zeros((self.nocc,self.nocc,self.nvirt,self.nvirt))
        _e = np.diag(self.dhf_fock_mo)
        for i in range(self.nocc):
            for j in range(self.nocc):
                for a in range(self.nvirt):
                    for b in range(self.nvirt):
                        self.D2[i,j,a,b] = -1./((_e[a+self.nocc] + _e[b+self.nocc] - _e[i] - _e[j]).real)

        _oovv = self.dhf_eri_full_asym[:self.nocc,:self.nocc,self.nocc:,self.nocc:]
        _vvoo = self.dhf_eri_full_asym[self.nocc:,self.nocc:,:self.nocc,:self.nocc]

        self.e_mp2 = 0.25*np.einsum('ijab,ijab,abij->',_oovv,self.D2,_vvoo)
        try:
            assert(abs(self.e_mp2.imag) < MACHEPS)
        except AssertionError:
            print(f'Imaginary part of MP2 energy is larger than {macheps}')
        
        _t1 = time.time()
        print(f'Relativistic MP2 Ecorr:      {self.e_mp2.real:15.7f} Eh')
        print(f'Relativistic MP2 Energy:     {(self.e_mp2 + self.dhf_e_rebuilt).real:15.7f} Eh')
        print(f'Time taken:                  {(_t1-_t0):15.7f} s\n')
        
    def run_casci(self, cas=None, do_fci=False, rdm_level=0):
        try:
            assert ((cas is None) and do_fci) or ((cas is not None) and (not do_fci))
        except AssertionError:
            raise Exception("If not doing FCI then a CAS must be provided via the 'cas' argument!")
            
        if cas is not None:
            try:
                assert type(cas) is tuple
            except AssertionError:
                raise Exception("'cas' must be a tuple of two integers!")
                
            try:
                assert int(cas[0]) <= int(cas[1])
            except AssertionError:
                raise Exception("Number of CAS electrons must be <= number of CAS spinors!")

        _t0 = time.time()
        
        if do_fci:
            _norbs = self.nlrg
            _nelec = self.nelec
            _ncoreel = 0
        else:
            _nelec, _norbs = cas
            _ncoreel = self.nelec-_nelec
            
        self.cas = (_nelec,_norbs)
        
        self.ncore = _ncoreel
        self.nact = _norbs
        self.nvirt = self.nlrg-(self.ncore+self.nact)
        self.nhole = self.ncore+self.nact
        self.npart = self.nact+self.nvirt
        
        self.core = slice(0,_ncoreel)
        self.active = slice(_ncoreel, _ncoreel+_norbs)
        self.virt = slice(_ncoreel+_norbs, self.nlrg)
        self.hole = slice(0,_ncoreel+_norbs)
        self.part = slice(_ncoreel, self.nlrg)
        
        self.hc = self.core
        self.ha = self.active
        self.pa = slice(0,self.nact)
        self.pv = slice(self.nact,self.nlrg)
        
        self.ncombs = scipy.special.comb(_norbs, _nelec, exact=True)
        self.det_strings = list(map(set_bit, list(itertools.combinations(range(_norbs),_nelec))))
        assert(len(self.det_strings) == self.ncombs)
        
        self.hamil_det = np.zeros((self.ncombs,self.ncombs), dtype='complex128')
        for i in range(self.ncombs):
            for j in range(i+1):
                exlvl = get_excitation_level(self.det_strings[i], self.det_strings[j])
                if (exlvl <= 2):
                    if (i==j):
                        occ = bstring_to_occ_vec(self.det_strings[i], _nelec, _norbs)
                        self.hamil_det[i,i] = 0
                        for iocc in occ:
                            self.hamil_det[i,i] += self.dhf_hcore_mo[iocc+_ncoreel,iocc+_ncoreel]
                            for jocc in occ:
                                self.hamil_det[i,i] += 0.5*self.dhf_eri_full_asym[iocc+_ncoreel,jocc+_ncoreel,iocc+_ncoreel,jocc+_ncoreel]
                    elif (exlvl == 1):
                        occ = bstring_to_occ_vec(self.det_strings[i], _nelec, _norbs)
                        conn, perm = get_excit_connection(self.det_strings[i], self.det_strings[j], exlvl, _nelec, _norbs)
                        self.hamil_det[i,j] = self.dhf_hcore_mo[conn[0],conn[1]]
                        for iocc in occ:
                            self.hamil_det[i,j] += self.dhf_eri_full_asym[conn[0]+_ncoreel, iocc+_ncoreel, conn[1]+_ncoreel, iocc+_ncoreel]

                        self.hamil_det[i,j] *= perm
                        self.hamil_det[j,i] = np.conjugate(self.hamil_det[i,j])
                    elif (exlvl == 2):
                        conn, perm = get_excit_connection(self.det_strings[i], self.det_strings[j], exlvl, _nelec, _norbs)
                        self.hamil_det[i,j] = perm*self.dhf_eri_full_asym[conn[0][0]+_ncoreel, conn[0][1]+_ncoreel, conn[1][0]+_ncoreel, conn[1][1]+_ncoreel]
                        self.hamil_det[j,i] = np.conjugate(self.hamil_det[i,j])         
        _t1 = time.time()
        
        self.casci_eigvals, self.casci_eigvecs = np.linalg.eigh(self.hamil_det)
        #self.fci_eigvals += self.nuclear_repulsion
        _t2 = time.time()
        
        if (rdm_level>0):
            try:
                assert rdm_level <= 3
            except AssertionError:
                raise Exception("RDM level up to 3 supported!")
                
            rdms = {'max_rdm_level':rdm_level}
            if (rdm_level>=1):
                rdms['1rdm'] = self.get_1_rdm((_nelec,_norbs), self.casci_eigvecs[:,0])
                self.fock = self.dhf_hcore_mo + np.einsum('piqi->pq',self.dhf_eri_full_asym[:,self.core,:,self.core]) + np.einsum('piqj,ij->pq',self.dhf_eri_full_asym[:,self.active,:,self.active],rdms['1rdm'])
            if (rdm_level>=2):
                if (_nelec>=2):
                    rdms['2rdm'] = self.get_2_rdm((_nelec,_norbs), self.casci_eigvecs[:,0])
                else:
                    rdms['2rdm'] = np.zeros((_norbs,_norbs,_norbs,_norbs), dtype='complex128')
            if (rdm_level>=3):
                if (_nelec>=3):
                    rdms['3rdm'] = self.get_3_rdm((_nelec,_norbs), self.casci_eigvecs[:,0])
                else:
                    rdms['3rdm'] = np.zeros((_norbs,_norbs,_norbs,_norbs,_norbs,_norbs), dtype='complex128')
                    
        _t3 = time.time()
        
        print(f'Relativistic CASCI({self.cas[0]},{self.cas[1]})')
        print(f'Eactive:                      {self.casci_eigvals[0].real:15.7f} Eh')
        
        if (rdm_level>=1):
            self.casci_ecore = 0
            self.casci_ecore += np.einsum('mm->',self.dhf_hcore_mo[self.core,self.core])
            self.casci_ecore += 0.5 * np.einsum('mnmn->',self.dhf_eri_full_asym[self.core,self.core,self.core,self.core])
            self.casci_ecore += np.einsum('mumv,uv->',self.dhf_eri_full_asym[self.core,self.active,self.core,self.active],rdms['1rdm'])
            print(f'Enuc:                         {self.nuclear_repulsion:15.7f} Eh')
            print(f'Ecore:                        {self.casci_ecore.real:15.7f} Eh')

        if (rdm_level>=2):
            _casci_eactive = np.einsum('uv,uv->',self.dhf_hcore_mo[self.active,self.active],rdms['1rdm'])
            _casci_eactive += 0.25 * np.einsum('uvxy,uvxy->',self.dhf_eri_full_asym[self.active,self.active,self.active,self.active],rdms['2rdm'])
            print(f'Eactive (built from RDM):     {_casci_eactive.real:15.7f} Eh')
            print(f'E0 (built from RDM):          {(_casci_eactive+self.casci_ecore).real+self.nuclear_repulsion:15.7f} Eh')
        
        if (rdm_level>=1):
            print(f'E0:                           {(self.casci_eigvals[0]+self.casci_ecore).real+self.nuclear_repulsion:15.7f} Eh')
            print(f'Ecorr:                        {((self.casci_eigvals[0]+self.casci_ecore).real+self.nuclear_repulsion-self.dhf_energy):15.7f} Eh')
        
        _t4 = time.time()
        print()
        print(f'Time taken:                   {(_t4-_t0):15.7f} s')
        print(f'... Hamil build:              {(_t1-_t0):15.7f} s')
        print(f'... Hamil diag:               {(_t2-_t1):15.7f} s')
        if (rdm_level > 0):
            print(f'... RDM build:                {(_t3-_t2):15.7f} s')
            return rdms
        
    def get_1_rdm(self, cas, psi):
        _t0 = time.time()
        _rdm = np.zeros((cas[1],cas[1]), dtype='complex128')
        for i in range(self.ncombs):
            occ_vec = bstring_to_occ_vec(self.det_strings[i], *cas)
            contrib = np.conjugate(psi[i])*psi[i]
            for p in occ_vec:
                _rdm[p, p] += contrib
            for j in range(self.ncombs):
                if (get_excitation_level(self.det_strings[i], self.det_strings[j]) == 1):
                    [[p], [q]], perm = get_excit_connection(self.det_strings[i], self.det_strings[j], 1, *cas)
                    _rdm[p, q] += perm*np.conjugate(psi[i])*psi[j]
        _t1 = time.time()
        print(f'Time taken for 1-RDM build:  {(_t1-_t0):15.7f} s\n')
        return _rdm
    
    def get_2_rdm(self, cas, psi):
        _t0 = time.time()
        _rdm = np.zeros((cas[1],cas[1],cas[1],cas[1]), dtype='complex128')
        for i in range(self.ncombs):
            # <p+ q+ q p>
            occ_vec = bstring_to_occ_vec(self.det_strings[i], *cas)
            # get all possible pairs of occupied spinorb
            contrib = np.conjugate(psi[i])*psi[i]
            for ip, p in enumerate(occ_vec):
                for q in occ_vec[:ip]:
                    _rdm[p, q, p, q] += contrib
                    _rdm[p, q, q, p] -= contrib
                    _rdm[q, p, p, q] -= contrib
                    _rdm[q, p, q, p] += contrib
            
            for j in range(self.ncombs):
                exlvl = get_excitation_level(self.det_strings[i], self.det_strings[j])
                
                if (exlvl==1):
                    # We need to accumulate all <p+ q+ q r>, it's sufficient to get the parity of <p+ r> since q+ q always cancel out
                    [[p],[r]], perm = get_excit_connection(self.det_strings[i], self.det_strings[j], 1, *cas)
                    f = annop(self.det_strings[i],p)[1]
                    occ_vec = bstring_to_occ_vec(f, cas[0]-1, cas[1])
                    contrib = perm*np.conjugate(psi[i])*psi[j]
                    for q in occ_vec:
                        _rdm[p, q, r, q] += contrib
                        _rdm[p, q, q, r] -= contrib
                        _rdm[q, p, r, q] -= contrib
                        _rdm[q, p, q, r] += contrib
                        
                        #_rdm[r, q, p, q] += np.conjugate(contrib)
                        #_rdm[q, r, p, q] -= np.conjugate(contrib)
                        #_rdm[r, q, q, p] -= np.conjugate(contrib)
                        #_rdm[q, r, q, p] += np.conjugate(contrib)
                elif (exlvl==2):
                    # <p+ q+ s r>
                    conn, perm = get_excit_connection(self.det_strings[i], self.det_strings[j], 2, *cas)
                    p, q = conn[0] # get_excit_connection's perm is <q+ p+ r s>
                    r, s = conn[1] # conn is in ascending order in spinor index, 
                    contrib = perm*np.conjugate(psi[i])*psi[j]
                    _rdm[p, q, r, s] += contrib
                    _rdm[p, q, s, r] -= contrib
                    _rdm[q, p, r, s] -= contrib
                    _rdm[q, p, s, r] += contrib
                    
                    #_rdm[r, s, p, q] += np.conjugate(contrib)
                    #_rdm[s, r, p, q] -= np.conjugate(contrib)
                    #_rdm[r, s, q, p] -= np.conjugate(contrib)
                    #_rdm[s, r, q, p] += np.conjugate(contrib)
        _t1 = time.time()
        print(f'Time taken for 2-RDM build:  {(_t1-_t0):15.7f} s\n')
        return _rdm
    
    def get_3_rdm(self, cas, psi):
        """
        gamma3^{pqr}_{stu} = <p+ q+ r+ u t s>
        """
        _t0 = time.time()
        _rdm = np.zeros((cas[1],cas[1],cas[1],cas[1],cas[1],cas[1]), dtype='complex128')
        for i in range(self.ncombs):
            occ_vec = bstring_to_occ_vec(self.det_strings[i], *cas)
            # get all possible triplets of occupied spinors
            contrib = np.conjugate(psi[i])*psi[i]
            for ip, p in enumerate(occ_vec):
                for iq, q in enumerate(occ_vec[:ip]):
                    for r in occ_vec[:iq]:
                        _rdm[p, q, r, p, q, r] += contrib
                        _rdm[q, r, p, p, q, r] += contrib
                        _rdm[r, p, q, p, q, r] += contrib
                        _rdm[p, r, q, p, q, r] -= contrib
                        _rdm[r, q, p, p, q, r] -= contrib
                        _rdm[q, p, r, p, q, r] -= contrib
                        
                        _rdm[p, q, r, q, r, p] += contrib
                        _rdm[q, r, p, q, r, p] += contrib
                        _rdm[r, p, q, q, r, p] += contrib
                        _rdm[p, r, q, q, r, p] -= contrib
                        _rdm[r, q, p, q, r, p] -= contrib
                        _rdm[q, p, r, q, r, p] -= contrib
                        
                        _rdm[p, q, r, r, p, q] += contrib
                        _rdm[q, r, p, r, p, q] += contrib
                        _rdm[r, p, q, r, p, q] += contrib
                        _rdm[p, r, q, r, p, q] -= contrib
                        _rdm[r, q, p, r, p, q] -= contrib
                        _rdm[q, p, r, r, p, q] -= contrib

                        _rdm[p, q, r, p, r, q] -= contrib
                        _rdm[q, r, p, p, r, q] -= contrib
                        _rdm[r, p, q, p, r, q] -= contrib
                        _rdm[p, r, q, p, r, q] += contrib
                        _rdm[r, q, p, p, r, q] += contrib
                        _rdm[q, p, r, p, r, q] += contrib
                        
                        _rdm[p, q, r, r, q, p] -= contrib
                        _rdm[q, r, p, r, q, p] -= contrib
                        _rdm[r, p, q, r, q, p] -= contrib
                        _rdm[p, r, q, r, q, p] += contrib
                        _rdm[r, q, p, r, q, p] += contrib
                        _rdm[q, p, r, r, q, p] += contrib
                        
                        _rdm[p, q, r, q, p, r] -= contrib
                        _rdm[q, r, p, q, p, r] -= contrib
                        _rdm[r, p, q, q, p, r] -= contrib
                        _rdm[p, r, q, q, p, r] += contrib
                        _rdm[r, q, p, q, p, r] += contrib
                        _rdm[q, p, r, q, p, r] += contrib
            
            for j in range(self.ncombs):
                exlvl = get_excitation_level(self.det_strings[i], self.det_strings[j])
                
                if (exlvl==1):
                    # We need to accumulate all <p+ q+ r+ r q s>, it's sufficient to get the parity of <p+ s> since q+r+ rq always cancel out
                    [[p], [s]], perm = get_excit_connection(self.det_strings[i], self.det_strings[j], 1, *cas)
                    f = annop(self.det_strings[i],p)[1]
                    occ_vec = bstring_to_occ_vec(f, cas[0]-1, cas[1])
                    contrib = perm*np.conjugate(psi[i])*psi[j]
                    for iq, q in enumerate(occ_vec):
                        # q cannot be == r as violates exclusion principle
                        for r in occ_vec[:iq]:
                            _rdm[p, q, r, s, q, r] += contrib
                            _rdm[q, r, p, s, q, r] += contrib
                            _rdm[r, p, q, s, q, r] += contrib
                            _rdm[p, r, q, s, q, r] -= contrib
                            _rdm[r, q, p, s, q, r] -= contrib
                            _rdm[q, p, r, s, q, r] -= contrib

                            _rdm[p, q, r, q, r, s] += contrib
                            _rdm[q, r, p, q, r, s] += contrib
                            _rdm[r, p, q, q, r, s] += contrib
                            _rdm[p, r, q, q, r, s] -= contrib
                            _rdm[r, q, p, q, r, s] -= contrib
                            _rdm[q, p, r, q, r, s] -= contrib

                            _rdm[p, q, r, r, s, q] += contrib
                            _rdm[q, r, p, r, s, q] += contrib
                            _rdm[r, p, q, r, s, q] += contrib
                            _rdm[p, r, q, r, s, q] -= contrib
                            _rdm[r, q, p, r, s, q] -= contrib
                            _rdm[q, p, r, r, s, q] -= contrib

                            _rdm[p, q, r, s, r, q] -= contrib
                            _rdm[q, r, p, s, r, q] -= contrib
                            _rdm[r, p, q, s, r, q] -= contrib
                            _rdm[p, r, q, s, r, q] += contrib
                            _rdm[r, q, p, s, r, q] += contrib
                            _rdm[q, p, r, s, r, q] += contrib

                            _rdm[p, q, r, r, q, s] -= contrib
                            _rdm[q, r, p, r, q, s] -= contrib
                            _rdm[r, p, q, r, q, s] -= contrib
                            _rdm[p, r, q, r, q, s] += contrib
                            _rdm[r, q, p, r, q, s] += contrib
                            _rdm[q, p, r, r, q, s] += contrib

                            _rdm[p, q, r, q, s, r] -= contrib
                            _rdm[q, r, p, q, s, r] -= contrib
                            _rdm[r, p, q, q, s, r] -= contrib
                            _rdm[p, r, q, q, s, r] += contrib
                            _rdm[r, q, p, q, s, r] += contrib
                            _rdm[q, p, r, q, s, r] += contrib
                if (exlvl==2):
                    # We need to accumulate all <p+ q+ r+ r t s>
                    conn, perm = get_excit_connection(self.det_strings[i], self.det_strings[j], 2, *cas)
                    p, q = conn[0] # get_excit_connection's perm is <q+ p+ r s>
                    s, t = conn[1] # conn is in ascending order in spinor index, 
                    f = annop_mult(self.det_strings[i],conn[0])[1]
                    occ_vec = bstring_to_occ_vec(f, cas[0]-2, cas[1])
                    contrib = perm*np.conjugate(psi[i])*psi[j]
                    for r in occ_vec:                       
                        _rdm[p, q, r, s, t, r] += contrib
                        _rdm[q, r, p, s, t, r] += contrib
                        _rdm[r, p, q, s, t, r] += contrib
                        _rdm[p, r, q, s, t, r] -= contrib
                        _rdm[r, q, p, s, t, r] -= contrib
                        _rdm[q, p, r, s, t, r] -= contrib

                        _rdm[p, q, r, t, r, s] += contrib
                        _rdm[q, r, p, t, r, s] += contrib
                        _rdm[r, p, q, t, r, s] += contrib
                        _rdm[p, r, q, t, r, s] -= contrib
                        _rdm[r, q, p, t, r, s] -= contrib
                        _rdm[q, p, r, t, r, s] -= contrib

                        _rdm[p, q, r, r, s, t] += contrib
                        _rdm[q, r, p, r, s, t] += contrib
                        _rdm[r, p, q, r, s, t] += contrib
                        _rdm[p, r, q, r, s, t] -= contrib
                        _rdm[r, q, p, r, s, t] -= contrib
                        _rdm[q, p, r, r, s, t] -= contrib

                        _rdm[p, q, r, s, r, t] -= contrib
                        _rdm[q, r, p, s, r, t] -= contrib
                        _rdm[r, p, q, s, r, t] -= contrib
                        _rdm[p, r, q, s, r, t] += contrib
                        _rdm[r, q, p, s, r, t] += contrib
                        _rdm[q, p, r, s, r, t] += contrib

                        _rdm[p, q, r, r, t, s] -= contrib
                        _rdm[q, r, p, r, t, s] -= contrib
                        _rdm[r, p, q, r, t, s] -= contrib
                        _rdm[p, r, q, r, t, s] += contrib
                        _rdm[r, q, p, r, t, s] += contrib
                        _rdm[q, p, r, r, t, s] += contrib

                        _rdm[p, q, r, t, s, r] -= contrib
                        _rdm[q, r, p, t, s, r] -= contrib
                        _rdm[r, p, q, t, s, r] -= contrib
                        _rdm[p, r, q, t, s, r] += contrib
                        _rdm[r, q, p, t, s, r] += contrib
                        _rdm[q, p, r, t, s, r] += contrib
                if (exlvl==3):
                    conn, perm = get_excit_connection(self.det_strings[i], self.det_strings[j], 3, *cas)
                    p, q, r = conn[0]
                    s, t, u = conn[1]
                    
                    contrib = perm*np.conjugate(psi[i])*psi[j]
                    
                    _rdm[p, q, r, s, t, r] += contrib
                    _rdm[q, r, p, s, t, r] += contrib
                    _rdm[r, p, q, s, t, r] += contrib
                    _rdm[p, r, q, s, t, r] -= contrib
                    _rdm[r, q, p, s, t, r] -= contrib
                    _rdm[q, p, r, s, t, r] -= contrib

                    _rdm[p, q, r, t, r, s] += contrib
                    _rdm[q, r, p, t, r, s] += contrib
                    _rdm[r, p, q, t, r, s] += contrib
                    _rdm[p, r, q, t, r, s] -= contrib
                    _rdm[r, q, p, t, r, s] -= contrib
                    _rdm[q, p, r, t, r, s] -= contrib

                    _rdm[p, q, r, r, s, t] += contrib
                    _rdm[q, r, p, r, s, t] += contrib
                    _rdm[r, p, q, r, s, t] += contrib
                    _rdm[p, r, q, r, s, t] -= contrib
                    _rdm[r, q, p, r, s, t] -= contrib
                    _rdm[q, p, r, r, s, t] -= contrib

                    _rdm[p, q, r, s, r, t] -= contrib
                    _rdm[q, r, p, s, r, t] -= contrib
                    _rdm[r, p, q, s, r, t] -= contrib
                    _rdm[p, r, q, s, r, t] += contrib
                    _rdm[r, q, p, s, r, t] += contrib
                    _rdm[q, p, r, s, r, t] += contrib

                    _rdm[p, q, r, r, t, s] -= contrib
                    _rdm[q, r, p, r, t, s] -= contrib
                    _rdm[r, p, q, r, t, s] -= contrib
                    _rdm[p, r, q, r, t, s] += contrib
                    _rdm[r, q, p, r, t, s] += contrib
                    _rdm[q, p, r, r, t, s] += contrib

                    _rdm[p, q, r, t, s, r] -= contrib
                    _rdm[q, r, p, t, s, r] -= contrib
                    _rdm[r, p, q, t, s, r] -= contrib
                    _rdm[p, r, q, t, s, r] += contrib
                    _rdm[r, q, p, t, s, r] += contrib
                    _rdm[q, p, r, t, s, r] += contrib
        _t1 = time.time()
        print(f'Time taken for 3-RDM build:  {(_t1-_t0):15.7f} s\n')
        return _rdm

In [3]:
def regularized_denominator(x, s):
    if abs(x) <= MACHEPS:
        return 0.0
    return (1. - np.exp(-s * x**2)) / x

def one_plus_exp_s(x):
    return 1. + np.exp(-s * x * x)

def exp_s(x):
    return np.exp(-s * x * x)

def dsrg_HT(F, V, T1, T2, gamma1, eta1, lambda2, lambda3, mf):
    # All three terms for MRPT2/3 are the same, but just involve different F/V
    hc = mf.core
    ha = mf.active

    pa = slice(0,mf.nact)
    pv = slice(mf.nact,mf.nlrg)

    # all quantities are stored ^{hh..}_{pp..}
    # h = {c,a}; p = {a, v}

    R = 0.0
    R += 1.000000000 * np.einsum("iu,iv,vu->",F[hc,pa],T1[hc,pa],eta1,optimize="optimal")
    R += -0.500000000 * np.einsum("iu,ixvw,vwux->",F[hc,pa],T2[hc,ha,pa,pa],lambda2,optimize="optimal")
    R += 1.000000000 * np.einsum("ia,ia->",F[hc,pv],T1[hc,pv],optimize="optimal")
    R += 1.000000000 * np.einsum("ua,va,uv->",F[ha,pv],T1[ha,pv],gamma1,optimize="optimal")
    R += -0.500000000 * np.einsum("ua,wxva,uvwx->",F[ha,pv],T2[ha,ha,pa,pv],lambda2,optimize="optimal")
    R += -0.500000000 * np.einsum("iu,ivwx,uvwx->",T1[hc,pa],V[hc,ha,pa,pa],lambda2,optimize="optimal")
    R += -0.500000000 * np.einsum("ua,vwxa,vwux->",T1[ha,pv],V[ha,ha,pa,pv],lambda2,optimize="optimal")
    R += 0.250000000 * np.einsum("ijuv,ijwx,vx,uw->",T2[hc,hc,pa,pa],V[hc,hc,pa,pa],eta1,eta1,optimize="optimal")
    R += 0.125000000 * np.einsum("ijuv,ijwx,uvwx->",T2[hc,hc,pa,pa],V[hc,hc,pa,pa],lambda2,optimize="optimal")
    R += 0.500000000 * np.einsum("iwuv,ixyz,vz,uy,xw->",T2[hc,ha,pa,pa],V[hc,ha,pa,pa],eta1,eta1,gamma1,optimize="optimal")
    R += 1.000000000 * np.einsum("iwuv,ixyz,vz,uxwy->",T2[hc,ha,pa,pa],V[hc,ha,pa,pa],eta1,lambda2,optimize="optimal")
    R += 0.250000000 * np.einsum("iwuv,ixyz,xw,uvyz->",T2[hc,ha,pa,pa],V[hc,ha,pa,pa],gamma1,lambda2,optimize="optimal")
    R += 0.250000000 * np.einsum("iwuv,ixyz,uvxwyz->",T2[hc,ha,pa,pa],V[hc,ha,pa,pa],lambda3,optimize="optimal")
    R += 0.500000000 * np.einsum("ijua,ijva,uv->",T2[hc,hc,pa,pv],V[hc,hc,pa,pv],eta1,optimize="optimal")
    R += 1.000000000 * np.einsum("ivua,iwxa,ux,wv->",T2[hc,ha,pa,pv],V[hc,ha,pa,pv],eta1,gamma1,optimize="optimal")
    R += 1.000000000 * np.einsum("ivua,iwxa,uwvx->",T2[hc,ha,pa,pv],V[hc,ha,pa,pv],lambda2,optimize="optimal")
    R += 0.500000000 * np.einsum("vwua,xyza,uz,yw,xv->",T2[ha,ha,pa,pv],V[ha,ha,pa,pv],eta1,gamma1,gamma1,optimize="optimal")
    R += 0.250000000 * np.einsum("vwua,xyza,uz,xyvw->",T2[ha,ha,pa,pv],V[ha,ha,pa,pv],eta1,lambda2,optimize="optimal")
    R += 1.000000000 * np.einsum("vwua,xyza,yw,uxvz->",T2[ha,ha,pa,pv],V[ha,ha,pa,pv],gamma1,lambda2,optimize="optimal")
    R += -0.250000000 * np.einsum("vwua,xyza,uxyvwz->",T2[ha,ha,pa,pv],V[ha,ha,pa,pv],lambda3,optimize="optimal")
    R += 0.250000000 * np.einsum("ijab,ijab->",T2[hc,hc,pv,pv],V[hc,hc,pv,pv],optimize="optimal")
    R += 0.500000000 * np.einsum("iuab,ivab,vu->",T2[hc,ha,pv,pv],V[hc,ha,pv,pv],gamma1,optimize="optimal")
    R += 0.250000000 * np.einsum("uvab,wxab,xv,wu->",T2[ha,ha,pv,pv],V[ha,ha,pv,pv],gamma1,gamma1,optimize="optimal")
    R += 0.125000000 * np.einsum("uvab,wxab,wxuv->",T2[ha,ha,pv,pv],V[ha,ha,pv,pv],lambda2,optimize="optimal")
    return R


In [4]:
s = 1.0
mol = pyscf.gto.M(
    verbose = 2,
    atom = '''
Li 0 0 0
Li 0 1.5 0
''',
    basis = 'sto-3g', spin=0, charge=0
)
a = mySCF(mol)
a.run_rhf()
a.run_dhf(rebuild=True, debug=True)
a.run_mp2()
rdms = a.run_casci(cas=(2,4), do_fci=False, rdm_level=3)
#rdms = a.run_casci(do_fci=True, rdm_level=2)

Non-relativistic RHF Energy:     -14.5306966 Eh
Relativistic DHF Energy:         -14.5318869 Eh
Rebuilt DHF Energy:              -14.5318869 Eh
Diff:           1.890044e-12
Diff 1e:        2.842171e-14
Diff 2e:        1.917577e-12

Timing report
....integral retrieval:            0.1048071 s
....integral transformation:       0.0754251 s
....integral contractions:         0.0011308 s
Total time taken:                  0.1814659 s

Relativistic MP2 Ecorr:           -0.0227703 Eh
Relativistic MP2 Energy:         -14.5546572 Eh
Time taken:                        0.0075085 s

Time taken for 1-RDM build:        0.0005550 s

Time taken for 2-RDM build:        0.0010083 s

Relativistic CASCI(2,4)
Eactive:                           -3.3430730 Eh
Enuc:                               3.1750633 Eh
Ecore:                            -14.3871490 Eh
Eactive (built from RDM):          -3.3430730 Eh
E0 (built from RDM):              -14.5551587 Eh
E0:                               -14.5551587 Eh
Ecorr: 

In [5]:
cumulants = make_cumulants(rdms)

In [6]:
fdiag = np.real(np.diagonal(a.fock))
d1 = np.zeros((a.nhole,a.npart),dtype='float64')
d2 = np.zeros((a.nhole,a.nhole,a.npart,a.npart),dtype='float64')
for i in range(a.nhole):
    for k in range(a.npart):
        d1[i,k] = regularized_denominator(fdiag[i]-fdiag[k+a.ncore], s)
        for j in range(a.nhole):
            for l in range(a.npart):
                d2[i,j,k,l] = regularized_denominator(fdiag[i]+fdiag[j]-fdiag[k+a.ncore]-fdiag[l+a.ncore], s)
                
denom_act = np.zeros((a.nact,a.nact),dtype='float64')
for i in range(a.nact):
    for j in range(a.nact):
        denom_act[i,j] = (fdiag[i+a.ncore]-fdiag[j+a.ncore])
        
d1_exp = np.zeros((a.nhole,a.npart),dtype='float64')
d2_exp = np.zeros((a.nhole,a.nhole,a.npart,a.npart),dtype='float64')
for i in range(a.nhole):
    for k in range(a.npart):
        d1_exp[i,k] = np.exp(-s*(fdiag[i]-fdiag[k+a.ncore])**2)
        for j in range(a.nhole):
            for l in range(a.npart):
                d2_exp[i,j,k,l] = np.exp(-s*(fdiag[i]+fdiag[j]-fdiag[k+a.ncore]-fdiag[l+a.ncore])**2)

In [7]:
T2_1 = np.conjugate(a.dhf_eri_full_asym[a.hole,a.hole,a.part,a.part].copy()) * d2

In [8]:
T1_1 = np.conjugate(a.fock[a.hole,a.part].copy())
T1_1 += np.einsum('xu,iuax,xu->ia', denom_act, T2_1[:,a.ha,:,a.pa],cumulants['gamma1'])
T1_1 *= d1

In [9]:
F_1_tilde = a.fock[a.hole,a.part].copy()
F_1_tilde += F_1_tilde * d1_exp
F_1_tilde += np.multiply(d1_exp, np.einsum('xu,iuax,xu->ia',denom_act,T2_1[:,a.ha,:,a.pa],cumulants['gamma1']))

In [10]:
V_1_tilde = a.dhf_eri_full_asym[a.hole,a.hole,a.part,a.part].copy()
V_1_tilde += V_1_tilde * d2_exp

In [11]:
dsrg_HT(F_1_tilde, V_1_tilde, T1_1, T2_1, cumulants['gamma1'], cumulants['eta1'], cumulants['lambda2'], cumulants['lambda3'],a)

(-0.005880745760484692+3.627726122078981e-15j)

Next, we compute the reference energy to test the integrals and the RDMs

We compute the energy as
$$
E = V_\mathrm{NN} + \sum_m^\mathrm{C} h_{mm} + \frac{1}{2} \sum_{mn}^\mathrm{C} \langle mn \| mn \rangle +
 \frac{1}{2} \sum_{m}^\mathrm{C} \langle mu \| mv \rangle \lambda^{v}_{u}
+ \sum_{uv}^\mathrm{A} h^{u}_{v} \lambda^{v}_{u}
+ \frac{1}{4} \sum_{uvxy}^\mathrm{A} \langle uv \| xy \rangle \left( \lambda^{x}_{u} \lambda^{y}_{v} - \lambda^{x}_{v} \lambda^{y}_{u} + \lambda^{xy}_{uv} \right)
$$